<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Advanced Compilation Options with H-Series

* [H-Series Hardware Compilation](#H-Series-Hardware-Compilation)
  * [Comnp.np.piling to H-Series Hardware Native Gates](#Comnp.piling-to-H-Series-Hardware-Native-Gates)
  * [The General SU(4) Entangler Gate](#The-General-SU(4)-Entangler-Gate)
  * [Controlling H-Series Hardware Compiler Optimizations](#Controlling-H-Series-Hardware-Compiler-Optimizations)
    * [Circuits written in any gate set](#Circuits-written-in-any-gate-set)
    * [Circuits written in the hardware's native gate set](#Circuits-written-in-the-hardware's-native-gate-set)
* [Examples](#Examples)
    * [1. Benchmarking Circuit in non-native gate set](#1.-Benchmarking-Circuit-in-non-native-gate-set)
    * [2. QFT Circuits in native gate set](#2.-QFT-Circuits-in-native-gate-set)
    * [3. Circuit using $SU(4)$](#Circuit-using-$SU(4)$)

## H-Series Hardware Compilation

Native gates are gates on a quantum computer that the hardware physically executes. Different quantum computers may have different gates that are physically executed on the hardware. Writing a gate in a quantum circuit doesn't guarantee its physical execution on the device. For instance, on H-Series quantum computers, a Hadamard gate written in the circuit is not the actual gate executed. When users submit circuits using a Hadamard gate, the gate is translated into a $U1q$ gate followed by a $Rz$ gate, which the ion trap device physically executes. See the *System Model H1 Product Data Sheet* on the [System Model H1](https://www.quantinuum.com/hardware/h1) page or the *System Model H2 Product Data Sheet* on the [System Model H2](https://www.quantinuum.com/hardware/h2) page for a listing of the H-Series hardware native gates. 

The H-Series hardware compiler handles the translation from circuits users submit to the native gates run on hardware. In the H-Series Quantum Charge-Coupled Device (QCCD) architecture, the hardware compilation includes the assignment of which physical qubit corresponds to which qubit in a circuit as well as how qubits will be transported around the device. Since transport, as well as gating, incurs a small amount of error with each operation, the H-Series compiler aims to minimize the number of gates that need to be executed. 

<div style="text-align: center;">
         <img src="figureshseries-compilation-stac6.png" width="400" />
</div>

### Compiling to H-Series Hardware Native Gates

On the Quantinuum H-Series devices, there are different native two-qubit gates available. The default native two-qubit gates include a fully entangling two-qubit gate, $ZZ()$, and an arbitrary angle ZZ gate, $Rzz(\theta)$. Another native gate available on the hardware is the General $SU(4)$ Entangler gate.

By default, the hardware compiler compiles to the $ZZ()$ or $Rzz(\theta)$ gate. Currently, only one native gate can be specified at a time. This ensures everything aligns in the global operations of the circuit. 

If users would like to use the General $SU(4)$ Entangler gate and not have the circuit rebased to $ZZ()$ or $Rzz(\theta)$ by the hardware compiler, they need to specify the $SU(4)$ gate using the `nativetq` option. The `nativetq` option is available to override the hardware stack's default two-qubit gate and use the supplied gate instead.

* `nativetq`: override the stack's default native two-qubit gate and use the supplied gate as the gate instead
  * `ZZ`: compile circuit to the $ZZ$ gate
  * `RZZ`: compile circuit to the $Rzz(\theta)$ gate, known as the `Optype.ZZPhase` gate within `pytket`.
  * `Rxxyyzz`: compile circuit to the $SU(4)$ gate, known as `Optype.TK2` within `pytket`.

### The General $SU(4)$ Entangler Gate

The General $SU(4)$ Entangler gate, or `Rxxyyzz`, is available in TKET as [`OpType.TK2`](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.TK2). This gate is a combination of `OpType.XXPhase`, `OpType.YYPhase` and `OpType.ZZPhase`, and requires three angles as input, $\alpha$, $\beta$ and $\gamma$. The definition of the gate is provided below:

$$\begin{equation} \textrm{TK2}(\alpha, \beta, \gamma) = e^{-\frac{1}{2} i \pi \alpha (\hat{X} \bigotimes \hat{X})} \quad e^{-\frac{1}{2} i \pi \beta (\hat{Y} \bigotimes \hat{Y})} \quad e^{-\frac{1}{2} \pi \gamma (\hat{Z} \bigotimes \hat{Z})} = e^{-\frac{1}{2} i \alpha (\hat{X} \bigotimes \hat{X}) + \beta (\hat{Y} \bigotimes \hat{Y}) + \gamma (\hat{Z} \bigotimes \hat{Z})} \end{equation} $$

This gate can be used as follows within TKET.

In [1]:
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import Circuit
from sympy import Symbol

symbols = [Symbol("a"), Symbol("b"), Symbol("c")]
circuit = Circuit(2)
circuit.TK2(*symbols, *circuit.qubits)
render_circuit_jupyter(circuit)

This circuit can be converted to a QASM string using the [`circuit_to_qasm_str`](https://tket.quantinuum.com/api-docs/qasm.html#pytket.qasm.circuit_to_qasm_str) function and by specifying the Quantinuum header `hqslib1`.

In [2]:
from pytket.qasm.qasm import circuit_to_qasm_str

print(circuit_to_qasm_str(circuit, header="hqslib1"))

OPENQASM 2.0;
include "hqslib1.inc";

qreg q[2];
Rxxyyzz((a)*pi,(b)*pi,(c)*pi) q[0],q[1];



### Controlling H-Series Hardware Compiler Optimizations

Users have the option of submitting circuits using whichever quantum gate set they desire. Users do not need to think about which physical gates will be executed or how physical qubits will move around the device since the hardware compiler manages this. In certain cases, however, users may want to know that the circuit they submit is going to be run on the device exactly as they write it. For example, when running benchmarking circuits users may want circuits to be executed exactly as specified in the circuit even if its not the most optimal in total number of two-qubit gates. 

Within the Quantinuum stack, the ability to control levels of TKET optimizations and control over what is executed on the hardware is provided between 4 different job submission parameters in the API.

There are two ways to think about using these options:
1. [Circuits written in any gate set](#Circuits-written-in-any-gate-set)
2. [Circuits written in the hardware's native gate set](#Circuits-written-in-the-hardware's-native-gate-set)

#### Circuits written in any gate set

Users are free to submit circuits written with any gate set, not just the native gate set of the hardware. In this case, the options for control over what optimizations are applied are given at the TKET level. TKET will rebase the circuit to the native gate set it believes is most optimal and the hardware compiler will handle further optimizations of gate combinations as it applies to transport and ion assignment. **We recommend this for the majority of use cases.**

* `tket-opt-level`: the `tket` optimization level to apply (default: `2`), with `tket` optimizations turned on, the hardware compiler will provide further gate combination logic as makes sense for ions and transport
  * `2`: powerful optimizations, can use approximate methods, compilation can be expensive
  * `1`: basic optimization, compiles quickly
  * `0`: rebase the circuit with `tket`
  * `null`: rebase the circuit without `tket`, using the hardware compiler only

#### Circuits written in the hardware's native gate set

For circuits that are written only using gates in the hardware's native gate set, various levels of control are provided for what optimizations will be performed in the stack.

Users may still choose to apply `tket` optimizations specified above to their circuit and control this with `tket-opt-level`, even if the circuit is written in the native gate set since further reductions in the number of quantum gates may be found, which will improve results. 

To use the native gates and circuit as the user submits it `tket-opt-level` must be set to `null` when submitting and the following options must be used.

* `no-opt`: turns off all `tket` optimizations *and* all hardware compiler gate combination logic. (default: `False`)
    * If more than 1 native gate is used in the circuit, the circuit will be rebased to 1 native gate, but no further gate combination logic will occur.
    * The job will fail if `no-opt` is set to `True` and the circuit contains non-native gates. 
* `noreduce`: turns off all `tket` optimizations, all hardware compiler gate combination logic, and requires exact 1:1 correspondence of two-qubit gates with gates on the system. This requires the circuit be submitted using one of the native two-qubit gates on the system, otherwise an error will be returned. (default: `False`)

Note that `tket-opt-level` set to `null`, `no-opt` set to `True`, and `noreduce` set to `True` all disable `tket` optimizations, but `no-opt` and `noreduce` also turn off all hardware gate combination logic.

**Create plot of quantum circuit -> tket opt or no-opt or nativetq or noreduce**

## Use-Case: Quantum Volume Test

### Quantum Volume Test Circuit

Quantum volume is a benchmarking test that was initially proposed by IBM [TODO: insert reference]. It is a test that aims to verify the quality as well as the quantity of qubits on the machine. The test does this by peforming rounds of single and two qubit gates between random pairs of qubits for as many rounds as qubits in the test. For example, for quantum volume of 2^N where N=4, 4 random rounds of gates are performed. If the quantum computer performing the test passes, it verifies that the quantum computer can perform quality computation with equal depth as width of the circuit. The advantage to using quantum volume is that it gives users the confidence that not only do they have the number of qubits to support running their circuit, but the two-qubit gate fidelity is there to support circuits of significant depth as well. Quantum algorithms need not only qubits, but the ability to run many gates consecutively as well. Quantinuum has steadily been increasing the quantum volume of H-Series machines. 

#### Decomposing a random SU(4) unitary into a circuit primitive

The function in the code-cell below defines a `pytket.circuit.CircBox`. This box contains a blueprint for the decomposition of a random SU(4) unitary distributed with the Haar Measure into a circuit primitive over 2-qubits. The implementation is based on [arxiv.0609050](http://arxiv.org/abs/math-ph/0609050). The random SU(4) unitary is generated using [`scipy.stats.unitary_group`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.unitary_group.html#scipy-stats-unitary-group).

In [3]:
from pytket.circuit import CircBox, Circuit, Unitary2qBox
import numpy as np
from scipy.stats import unitary_group

def haar_random_su4_box() -> CircBox:
    r = unitary_group.rvs(dim=4)
    circuit = Circuit(2)
    box = Unitary2qBox(r)
    circuit.add_unitary2qbox(box, 0, 1)
    return CircBox(circuit)

In [4]:
from pytket.circuit.display import render_circuit_jupyter

circuit = haar_random_su4_box().get_circuit()
render_circuit_jupyter(circuit)

The `pytket` sequence pass, consists of two passes:
* [`pytket.passes.DecomposeBoxes`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.DecomposeBoxes)
* [`pytket.passes.KAKDecomposition`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.KAKDecomposition)

Both passes are argument to [`pytket.passes.SequencePass`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.SequencePass). `SequencePass` allows both passes to be applied to the input circuit with one `apply` call.

In [5]:
from pytket.passes import DecomposeBoxes, KAKDecomposition, SequencePass
from pytket.circuit import OpType

sequence_pass = SequencePass([DecomposeBoxes(), KAKDecomposition(target_2qb_gate=OpType.TK2)])

In [6]:
sequence_pass.apply(circuit)
render_circuit_jupyter(circuit)

#### Building the Quantum Volume Test Circuit

An $n$-qubit Quantum Volume Test (QVT) consists of $n$ rounds of parallel SU(4) sequences. Each sequence is partitioned by permutations on the qubit IDs. In the code-cell below, the `haar_random_su4_box` is used to apply SU(4) unitaries.

In [7]:
circuit = Circuit(4)
for _ in range(4):
    permutation = np.random.permutation(circuit.qubits)
    for i in range(0, 4, 2):
        box = haar_random_su4_box()
        circuit.add_circbox(box, [permutation[i], permutation[i+1]])
circuit.measure_all()

[CircBox q[0], q[1]; CircBox q[2], q[3]; CircBox q[0], q[3]; CircBox q[2], q[1]; CircBox q[0], q[1]; CircBox q[2], q[3]; CircBox q[0], q[1]; CircBox q[2], q[3]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; Measure q[2] --> c[2]; Measure q[3] --> c[3]; ]

The `SequencePass` defined in the code-cell in the previous subsection is used below on the QVT circuit

In [8]:
sequence_pass.apply(circuit)
from pytket.circuit.display import render_circuit_jupyter
render_circuit_jupyter(circuit)

### Define default native two-qubit gate with `QuantinuumBackend`

#### SU(4) gate - OpType.TK2 (Rxxyyzz)

In order to simulate a circuit containing the native SU(4) gate on a H-Series device or emulator, the `QuantinuumBackendCompilationConfig` needs to specify that `OpType.TK2` is the native two-qubit to use by default. In addition, the `tket-op-level` L4 option needs to be set to None. This disables any additional tket optimisation in the stack

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend
from pytket.extensions.quantinuum.backends.quantinuum import QuantinuumBackendCompilationConfig

from pytket.circuit import OpType

compilation_config = QuantinuumBackendCompilationConfig(allow_implicit_swaps=True, target_2qb_gate=OpType.TK2)
compilation_config.target_2qb_gate

quantinuum_backend = QuantinuumBackend(device_name="H1-1", compilation_config=compilation_config, options={"tket-op-level": None})
quantinuum_backend.login()

In [10]:
quantinuum_backend.backend_info.gate_set

{<OpType.Barrier: 8>,
 <OpType.WASM: 14>,
 <OpType.SetBits: 15>,
 <OpType.CopyBits: 16>,
 <OpType.RangePredicate: 17>,
 <OpType.ExplicitPredicate: 18>,
 <OpType.ExplicitModifier: 19>,
 <OpType.MultiBit: 20>,
 <OpType.Rz: 36>,
 <OpType.TK2: 41>,
 <OpType.Measure: 63>,
 <OpType.Reset: 65>,
 <OpType.PhasedX: 68>,
 <OpType.ZZMax: 70>,
 <OpType.ZZPhase: 73>,
 <OpType.ClassicalExpBox: 103>}

In [12]:
qv_circuit_wSU4 = quantinuum_backend.get_compiled_circuit(circuit, optimisation_level=2)
qv_circuit_wSU4.name = "QV Example with SU(4)"
print(f"Number of TK2 (Rxxyyzz) Gates: {qv_circuit_wSU4.n_2qb_gates()}")
render_circuit_jupyter(qv_circuit_wSU4)

Number of TK2 (Rxxyyzz) Gates: 18


In [13]:
cost = quantinuum_backend.cost(qv_circuit_wSU4, n_shots=100, syntax_checker="H1-1SC")
print(f"SU(4) circuit cost: {cost} HQC)

SyntaxError: unterminated string literal (detected at line 2) (2097166687.py, line 2)

In [ ]:
handle = quantinuum_backend.process_circuit(qv_circuit_wSU4, n_shots=100)

In [ ]:
result = quantinuum_backend.get_result(handle)
print(result.get_distribution())

ConnectionClosedError: sent 1011 (internal error) keepalive ping timeout; no close frame received

#### Arbitrary Angle ZZ Gate - OpType.ZZPhase (Rzz)

In [ ]:
quantinuum_backend.set_compilation_config_target_2qb_gate(OpType.ZZPhase)

In [ ]:
qv_circuit_no_SU4 = quantinuum_backend.get_compiled_circuit(circuit, optimisation_level=2)
qv_circuit_no_SU4.name = "QV Example without SU(4)"
print(f"Number of ZZPhase Gates: {qv_circuit_no_SU4.n_2qb_gates()}")
render_circuit_jupyter(qv_circuit_no_SU4)

[U3(0.800268, 0.516276, 1.5618) q[0]; U3(2.90919, 0.444819, 1.87851) q[1]; U3(0.834017, 0.231266, 0.933179) q[2]; U3(1.30676, 0.506151, 0.356742) q[3]; U3(2.99557, 0.429204, 1.5708) q[0]; U3(2.99557, 0.429204, 1.5708) q[1]; U3(2.99557, 0.429204, 1.5708) q[2]; U3(2.99557, 0.429204, 1.5708) q[3]; U3(0, 0.429204, 0.712389) q[0]; U3(0.712389, 0, 0.712389) q[1]; U3(1.5708, 0.429204, 1.5708) q[2]; U3(1.5708, 0.429204, 1.5708) q[3]; U3(3.14159, 0.429204, 1.5708) q[0]; CX q[1], q[3]; U3(0.712389, 0, 0.712389) q[0]; U3(2.99557, 0.697067, 1.5708) q[1]; U3(3.14159, 0, 0.14751) q[3]; CX q[0], q[2]; CX q[1], q[3]; U3(2.99557, 0.601965, 1.5708) q[0]; U3(1.5708, 0, 1.5708) q[1]; U3(3.14159, 0, 1.92842) q[2]; U3(0, 0.429204, 0.0334044) q[3]; CX q[0], q[2]; CX q[1], q[3]; U3(1.5708, 0, 1.5708) q[0]; U3(1.5708, 0.429204, 1.5708) q[1]; U3(0, 0.429204, 0.0921141) q[2]; U3(1.5708, 0.429204, 1.5708) q[3]; CX q[0], q[2]; U3(0.802236, 1.74342, 0.729375) q[1]; U3(1.33296, 0.821419, 0.944898) q[3]; U3(3.14159, 

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>